In [1]:
#running PIPR from here, shell script will not detect CUDA devices 
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


import sys
if '../../embeddings' not in sys.path:
    sys.path.append('../../embeddings')

from seq2tensor import s2t
import keras

from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Embedding, LSTM, Bidirectional, BatchNormalization, add
from keras.layers.core import Flatten, Reshape
from keras.layers.merge import Concatenate, concatenate, subtract, multiply
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPooling1D, AveragePooling1D, GlobalAveragePooling1D

from keras.optimizers import Adam, RMSprop

import os
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

def get_session(gpu_fraction=0.75):
    '''Assume that you have 6GB of GPU memory and want to allocate ~2GB'''

    num_threads = os.environ.get('OMP_NUM_THREADS')
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction)

    if num_threads:
        return tf.Session(config=tf.ConfigProto(
            gpu_options=gpu_options, intra_op_parallelism_threads=num_threads))
    else:
        return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

KTF.set_session(get_session())

import numpy as np
from tqdm import tqdm

from keras.layers import Input, CuDNNGRU
from numpy import linalg as LA
import scipy

Using TensorFlow backend.


In [2]:
# change
id2seq_file = '../../sun/preprocessed/SEQ-Supp-ABCD.tsv'

id2index = {}
seqs = []
index = 0
for line in open(id2seq_file):
    line = line.strip().split('\t')
    id2index[line[0]] = index
    seqs.append(line[1])
    index += 1
seq_array = []
id2_aid = {}
sid = 0

seq_size = 600
emb_files = ['../../embeddings/default_onehot.txt', '../../embeddings/string_vec5.txt', '../../embeddings/CTCoding_onehot.txt', '../../embeddings/vec7_CTC.txt']
use_emb = 0
hidden_dim = 25
n_epochs=50

# ds_file, label_index, rst_file, use_emb, hidden_dim
ds_file = '../../yest/preprocessed/Supp-AB.tsv'
label_index = 2
rst_file = 'results/15k_onehot_cnn.txt'
sid1_index = 0
sid2_index = 1
'''
if len(sys.argv) > 1:
    ds_file, label_index, rst_file, use_emb, hidden_dim, n_epochs = sys.argv[1:]
    label_index = int(label_index)
    use_emb = int(use_emb)
    hidden_dim = int(hidden_dim)
    n_epochs = int(n_epochs)
'''
#manually set these for the three diff networks tested 
#../../../sun/preprocessed/Supp-AB.tsv -1 results/sun_wvctc_rcnn_25_5.txt 3 25 100
#../../../sun/preprocessed/Supp-AB.tsv -1 results/sun_wvctc_rcnn_50_5.txt 3 50 100
#../../../sun/preprocessed/Supp-AB.tsv -1 results/sun_wvctc_rcnn_75_5.txt 3 75 100
#best from the paper was hiden dim 50 
ds_file = "../../sun/preprocessed/Supp-AB.tsv"
label_index = -1
rst_file = "results/sun_wvctc_rcnn_50_5.txt"
use_emb = 3
hidden_dim = 50
n_epochs = 50
seq2t = s2t(emb_files[use_emb])

In [3]:


max_data = -1
limit_data = max_data > 0
raw_data = []
skip_head = True
x = None
count = 0

for line in tqdm(open(ds_file)):
    if skip_head:
        skip_head = False
        continue
    line = line.rstrip('\n').rstrip('\r').split('\t')
    if id2index.get(line[sid1_index]) is None or id2index.get(line[sid2_index]) is None:
        continue
    if id2_aid.get(line[sid1_index]) is None:
        id2_aid[line[sid1_index]] = sid
        sid += 1
        seq_array.append(seqs[id2index[line[sid1_index]]])
    line[sid1_index] = id2_aid[line[sid1_index]]
    if id2_aid.get(line[sid2_index]) is None:
        id2_aid[line[sid2_index]] = sid
        sid += 1
        seq_array.append(seqs[id2index[line[sid2_index]]])
    line[sid2_index] = id2_aid[line[sid2_index]]
    raw_data.append(line)
    if limit_data:
        count += 1
        if count >= max_data:
            break
print (len(raw_data))


len_m_seq = np.array([len(line.split()) for line in seq_array])
avg_m_seq = int(np.average(len_m_seq)) + 1
max_m_seq = max(len_m_seq)
print (avg_m_seq, max_m_seq)

dim = seq2t.dim
seq_tensor = np.array([seq2t.embed_normalized(line, seq_size) for line in tqdm(seq_array)])

seq_index1 = np.array([line[sid1_index] for line in tqdm(raw_data)])
seq_index2 = np.array([line[sid2_index] for line in tqdm(raw_data)])

print(seq_index1[:10])

class_map = {'0':1,'1':0}
print(class_map)
class_labels = np.zeros((len(raw_data), 2))
for i in range(len(raw_data)):
    class_labels[i][class_map[raw_data[i][label_index]]] = 1.

def build_model():
    seq_input1 = Input(shape=(seq_size, dim), name='seq1')
    seq_input2 = Input(shape=(seq_size, dim), name='seq2')
    l1=Conv1D(hidden_dim, 3)
    r1=Bidirectional(CuDNNGRU(hidden_dim, return_sequences=True))
    l2=Conv1D(hidden_dim, 3)
    r2=Bidirectional(CuDNNGRU(hidden_dim, return_sequences=True))
    l3=Conv1D(hidden_dim, 3)
    r3=Bidirectional(CuDNNGRU(hidden_dim, return_sequences=True))
    l4=Conv1D(hidden_dim, 3)
    r4=Bidirectional(CuDNNGRU(hidden_dim, return_sequences=True))
    l5=Conv1D(hidden_dim, 3)
    r5=Bidirectional(CuDNNGRU(hidden_dim, return_sequences=True))
    l6=Conv1D(hidden_dim, 3)
    s1=MaxPooling1D(3)(l1(seq_input1))
    s1=concatenate([r1(s1), s1])
    s1=MaxPooling1D(3)(l2(s1))
    s1=concatenate([r2(s1), s1])
    s1=MaxPooling1D(2)(l3(s1))
    s1=concatenate([r3(s1), s1])
    s1=MaxPooling1D(2)(l4(s1))
    s1=concatenate([r4(s1), s1])
    s1=MaxPooling1D(2)(l5(s1))
    s1=concatenate([r5(s1), s1])
    s1=l6(s1)
    s1=GlobalAveragePooling1D()(s1)
    s2=MaxPooling1D(3)(l1(seq_input2))
    s2=concatenate([r1(s2), s2])
    s2=MaxPooling1D(3)(l2(s2))
    s2=concatenate([r2(s2), s2])
    s2=MaxPooling1D(2)(l3(s2))
    s2=concatenate([r3(s2), s2])
    s2=MaxPooling1D(2)(l4(s2))
    s2=concatenate([r4(s2), s2])
    s2=MaxPooling1D(2)(l5(s2))
    s2=concatenate([r5(s2), s2])
    s2=l6(s2)
    s2=GlobalAveragePooling1D()(s2)
    merge_text = multiply([s1, s2])
    x = Dense(hidden_dim, activation='linear')(merge_text)
    x = keras.layers.LeakyReLU(alpha=0.3)(x)
    x = Dense(int((hidden_dim+7)/2), activation='linear')(x)
    x = keras.layers.LeakyReLU(alpha=0.3)(x)
    main_output = Dense(2, activation='softmax')(x)
    merge_model = Model(inputs=[seq_input1, seq_input2], outputs=[main_output])
    return merge_model

batch_size1 = 512


from sklearn.model_selection import KFold, ShuffleSplit
kf = KFold(n_splits=10, shuffle=True)
tries = 3
cur = 0
recalls = []
accuracy = []
total = []
total_truth = []
train_test = []
for train, test in kf.split(class_labels):
    if np.sum(class_labels[train], 0)[0] > 0.8 * len(train) or np.sum(class_labels[train], 0)[0] < 0.2 * len(train):
        continue
    train_test.append((train, test))
    cur += 1
    if cur >= tries:
        break

print (len(train_test))

#copy below
num_hit = 0.
num_total = 0.
num_pos = 0.
num_true_pos = 0.
num_false_pos = 0.
num_true_neg = 0.
num_false_neg = 0.
fold = 0
modelName = "modelHiddenDims" + str(hidden_dim)
for train, test in train_test:
    merge_model = None
    adam = Adam(lr=0.001, amsgrad=True, epsilon=1e-6)
    rms = RMSprop(lr=0.001)
    merge_model = build_model()
    merge_model.compile(optimizer=rms, loss='categorical_crossentropy', metrics=['accuracy'])
    merge_model.fit([seq_tensor[seq_index1[train]], seq_tensor[seq_index2[train]]], class_labels[train], batch_size=batch_size1, epochs=n_epochs)
    #result1 = merge_model.evaluate([seq_tensor1[test], seq_tensor2[test]], class_labels[test])
    pred = merge_model.predict([seq_tensor[seq_index1[test]], seq_tensor[seq_index2[test]]])
    for i in range(len(class_labels[test])):
        num_total += 1
        if np.argmax(class_labels[test][i]) == np.argmax(pred[i]):
            num_hit += 1
        if class_labels[test][i][0] > 0.:
            num_pos += 1.
            if pred[i][0] > pred[i][1]:
                num_true_pos += 1
            else:
                num_false_neg += 1
        else:
            if pred[i][0] > pred[i][1]:
                num_false_pos += 1
            else:
                num_true_neg += 1
    accuracy = num_hit / num_total
    prec = num_true_pos / (num_true_pos + num_false_pos)
    recall = num_true_pos / num_pos
    spec = num_true_neg / (num_true_neg + num_false_neg)
    f1 = 2. * prec * recall / (prec + recall)
    mcc = (num_true_pos * num_true_neg - num_false_pos * num_false_neg) / ((num_true_pos + num_true_neg) * (num_true_pos + num_false_neg) * (num_false_pos + num_true_neg) * (num_false_pos + num_false_neg)) ** 0.5
    print (accuracy, prec, recall, spec, f1, mcc)
    totalMName = modelName + "fold" + str(fold)
    fold += 1
    merge_model.save(totalMName)

accuracy = num_hit / num_total
prec = num_true_pos / (num_true_pos + num_false_pos)
recall = num_true_pos / num_pos
spec = num_true_neg / (num_true_neg + num_false_neg)
f1 = 2. * prec * recall / (prec + recall)
mcc = (num_true_pos * num_true_neg - num_false_pos * num_false_neg) / ((num_true_pos + num_true_neg) * (num_true_pos + num_false_neg) * (num_false_pos + num_true_neg) * (num_false_pos + num_false_neg)) ** 0.5
print (accuracy, prec, recall, f1)

with open(rst_file, 'w') as fp:
    fp.write('acc=' + str(accuracy) + '\tprec=' + str(prec) + '\trecall=' + str(recall) + '\tspec=' + str(spec) + '\tf1=' + str(f1) + '\tmcc=' + str(mcc))


73109it [00:00, 226348.86it/s]
  2%|▏         | 237/10364 [00:00<00:04, 2357.55it/s]

73108
2 1


100%|██████████| 73108/73108 [00:00<00:00, 987454.40it/s]


[ 0  2  4  6  8 10 12 14 16 18]
{'0': 1, '1': 0}
3






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/50




65797/65797 [==============================] - 62s 946us/step - loss: 0.7228 - acc: 0.5236
Epoch 2/50
65797/65797 [==============================] - 57s 865us/step - loss: 0.6947 - acc: 0.5331
Epoch 3/50
65797/65797 [==============================] - 57s 866us/step - loss: 0.6895 - acc: 0.5556
Epoch 4/50
65797/65797 [==============================] - 57s 865us/step - loss: 0.6792 - acc: 0.5952
Epoch 5/50
65797/65797 [==============================] - 57s 866us/step - loss: 0.6083 - acc: 0.6716
Epoch 6/50
65797/65797 [==============================] - 57s 865us/step - loss: 0.3612 - acc: 0.8478
Epoch 7/50
65797/65797 [==============================] - 57s 864us/step - loss: 0.2299 - acc: 0.9122
Epoch 8/50
65797/65797 [==============================] - 57s 863us/step - loss: 0.1764 - acc: 0.9386
Epoch 9/50
65797/65797 [===

Epoch 1/50
65797/65797 [==============================] - 60s 916us/step - loss: 0.8219 - acc: 0.5442
Epoch 2/50
65797/65797 [==============================] - 57s 863us/step - loss: 0.6910 - acc: 0.5562
Epoch 3/50
65797/65797 [==============================] - 57s 866us/step - loss: 0.6818 - acc: 0.5733
Epoch 4/50
65797/65797 [==============================] - 57s 866us/step - loss: 0.6733 - acc: 0.6066
Epoch 5/50
65797/65797 [==============================] - 57s 865us/step - loss: 0.5444 - acc: 0.7290
Epoch 6/50
65797/65797 [==============================] - 57s 864us/step - loss: 0.3133 - acc: 0.8730
Epoch 7/50
65797/65797 [==============================] - 57s 862us/step - loss: 0.2053 - acc: 0.9248
Epoch 8/50
65797/65797 [==============================] - 57s 866us/step - loss: 0.1535 - acc: 0.9465
Epoch 9/50
65797/65797 [==============================] - 57s 870us/step - loss: 0.1207 - acc: 0.9600
Epoch 10/50
65797/65797 [==============================] - 57s 865us/step - loss: 

FileNotFoundError: [Errno 2] No such file or directory: 'results/sun_wvctc_rcnn_50_5.txt'

In [ ]:
#fold 0
0.981261113390781 0.9871687587168759 0.974931129476584 0.975577026301664 0.9810117810117811 3.548676785606872
#fold 1
#0.9809191629052113 0.9840549828178694 0.9777383228626059 0.9778140737716075 0.9808864835240118 3.5152854613623807
#fold 2
#0.9821729813523001 0.9859452507808194 0.9783064442621456 0.9784556893274192 0.9821109941895045 3.6439374474318322
0.9821729813523001 0.9859452507808194 0.9783064442621456 0.9821109941895045